In [1]:
# import pandas as pd
import numpy as np
np.random.seed(42)
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from bokeh.plotting import figure, output_file, output_notebook, show 
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, ColumnDataSource, Legend, Band
from bokeh.models import BoxAnnotation, Toggle
from sklearn.mixture import GaussianMixture
# import imgkit
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', 200)
# pd.set_option('colheader_justify', 'center')
display(HTML("<h1>" + "Gminy Karpat" + "</h1>"))

/tmp/ipykernel_658453/1833454850.py:8: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
!pip install -U scikit-learn



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip available: 22.1.2 -> 22.2
[notice] To update, run: pip install --upgrade pip


In [3]:
wszystkie_gminy = pd.read_csv("GminyKarpatyWCalosciProcentParkuN.csv")
park_tresh = 5
wszystkie_gminy["is_park"] = wszystkie_gminy["ProcentGminyObjetyParkiemN"].apply(lambda x: True if x >= park_tresh else False)

wszystkie_gminy = wszystkie_gminy[wszystkie_gminy["Wojewodztwo"].isin([12,18])]
kody = wszystkie_gminy["Kod"].values #185 len
kanalizacja = pd.read_csv("kanalizacja.csv")
dochod_wlasny = pd.read_csv("dochod-wlasny.csv")
gestosc = pd.read_csv("gestosc.csv")

In [4]:
year = "2020"

gminy_wyrzucone = [1217011,#Zakopane
                    1821052,#Solina Zespół Elektrowni Wodnych Solina-Myczkowce
                   1817052,#Sanok zabytkowe miasto powiatowe
                   1210113,#Muszyna
                   1211023,#Szczawnica
                   1210073,#Krynica-Zdrój
                   1210133,#Piwniczna-Zdrój
                   1211092,#Nowy Targ
                   1211032,#Czarny Dunajec
                  ]

#Stryszów, Gnojnik, Trzciana, Chmielnik, Pcim, Lanckorona gminy które wadzą
wszystkie_gminy = wszystkie_gminy[~wszystkie_gminy["Kod"].isin(gminy_wyrzucone)]
is_park_code = wszystkie_gminy[wszystkie_gminy["is_park"]]["Kod"].values

gestosc = gestosc[gestosc["Kod"].isin(kody)]
max_density_park = gestosc[gestosc["Kod"].isin(is_park_code)].sort_values(by=[year])[year].values[-1]#140 poronin, najwyższa gęstość zaludnienia

for i in range(1,11):
    if max_density_park<gestosc[year].quantile(i/10):
        q = i/10
        q_val = gestosc[year].quantile(i/10)
        break
percentyle_60 = gestosc[gestosc[year] <= q_val]["Kod"].values
# no_park_code = wszystkie_gminy[(wszystkie_gminy["Kod"].isin(percentyle_60)) & (~wszystkie_gminy["is_park"])]

In [5]:
ludnosc_ogolem = pd.read_csv("ludnosc-ogolem.csv")
ludnosc_prod= pd.read_csv("ludnosc-prod.csv")
bezrobotni = pd.read_csv("bezrobotni.csv")
ludnosc_ogolem = ludnosc_ogolem[ludnosc_ogolem["Kod"].isin(percentyle_60)].sort_values(by=['Kod'])
ludnosc_prod = ludnosc_prod[ludnosc_prod["Kod"].isin(percentyle_60)].sort_values(by=['Kod'])
bezrobotni = bezrobotni[bezrobotni["Kod"].isin(percentyle_60)].sort_values(by=['Kod'])
kanalizacja = kanalizacja[kanalizacja["Kod"].isin(percentyle_60)].sort_values(by=['Kod'])
for i in ludnosc_ogolem["Kod"].values == ludnosc_prod["Kod"].values:
    if i == False: 
        raise Exception("Not sorted")
        break
for i in ludnosc_ogolem["Kod"].values == bezrobotni["Kod"].values:
    if i == False: 
        raise Exception("Not sorted")
        break
bezrobotni_p = pd.DataFrame(ludnosc_ogolem[["Kod","Nazwa"]])
for y in range(2003,2021):
    bezrobotni_p[str(y)] =  bezrobotni[str(y)].values/ludnosc_ogolem[str(y)].values*100

lasy = pd.read_csv("GminyKarpatyParkiLasy.csv")[["Nazwa","Kod","UdzialLasu"]] 
# ---------------------
wszystkie_gminy = wszystkie_gminy[wszystkie_gminy["Kod"].isin(percentyle_60)]
data = wszystkie_gminy[["Kod","Nazwa","ProcentGminyObjetyParkiemN","is_park"]]
gestosc = gestosc[["Kod", year]]
data = data.merge(gestosc[["Kod", year]], left_on='Kod', right_on='Kod')
data = data.rename(columns={year:"gestosc"})
data = data.merge(dochod_wlasny[["Kod", year]], left_on='Kod', right_on='Kod')
data = data.rename(columns={year:"dochod"})
data = data.merge(bezrobotni_p[["Kod", year]], left_on='Kod', right_on='Kod')
data = data.rename(columns={year:"bezrobotni_p"})

data = data.merge(lasy[["Kod", "UdzialLasu"]], left_on='Kod', right_on='Kod')


In [6]:
# np.random.seed(24)#no
np.random.seed(42)
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler 

def fix_nums(x_):
    if x_ == 0:
        return 1
    elif x_ == 1:
        return 0
    elif x_ == 2:
        return 2
scaler = MinMaxScaler()
X = data[["gestosc","UdzialLasu"]].values
X = scaler.fit_transform(X)
model = GaussianMixture(n_components=3)

model.fit(X)

data["clst"] = model.predict(X)
data["clst"] = data["clst"].apply(lambda x: fix_nums(x) )
clusters = data[["Kod","clst"]].groupby("clst")
groups = []
for name, group in clusters:

    groups.append(group.Kod.values)

fig = px.scatter(data, x="gestosc", y="UdzialLasu",
                 color="clst",
#                 color="is_park",
                  hover_name=data["Nazwa"],
                 title="Analiza zgrupowań: gminy w przestrzeni procent powierzchni zajmowanej przez las- gęstość zaludnienia")

 
for trace in fig['data']: 
    if(trace['name'] != 'B'): trace['showlegend'] = False
    trace.hovertemplate = trace.hovertemplate.replace("=", ": ")

# fig.show()

# matrix = data.corr().round(2)
# sns.heatmap(matrix, annot=True, cmap="vlag")
# plt.title("Macierz korelacji")
# plt.show()

titles = [" o najwyższym udziale lasów na swoim terenie.",
          " o średnim udziale lasów na swoim terenie.",
          " o najniższym udziale lasów na swoim terenie."]

In [7]:
fig = px.scatter(data, x="gestosc", y="dochod",
                 color="is_park",
#                 color="is_park",
                  hover_name=data["Nazwa"],
                 title="gęstość zaludnienia- dochod"
                )
# fig.show()

fig = px.scatter(data, x="UdzialLasu", y="dochod",
                 color="is_park",
#                 color="is_park",
                  hover_name=data["Nazwa"],
                 title="procent powierzchni zajmowanej przez las- dochod"
                )
# fig.show()

In [8]:
gestosc_wybrane = gestosc[gestosc["Kod"].isin(percentyle_60)]
dochod_wybrane = dochod_wlasny[dochod_wlasny["Kod"].isin(percentyle_60)]

In [9]:
display(HTML("<h1>" + "Gminy Dochody per capita raczej nie do pokazywania" + "</h1>"))


dochod_wybrane_clean = dochod_wybrane.dropna()
timespan = range(2002,2021)

wazne_gminy = [1801083,#Ustrzyki Dolne
               1813012,#Bircza
               1813032]#Fredropol
wazne_gminy_col = {1801083:"turquoise", 1813012:"coral",1813032:"darkseagreen"}

kurs = pd.read_csv("kurs_euro.csv")
kurs = kurs.groupby("year").agg("mean")
kurs["year"] = kurs.index
kurs = kurs[kurs["year"].isin(timespan)][["year","price"]]
kurs["grad"] = np.gradient(kurs.price.values)
kurs["rosnie"] = kurs["grad"].apply(lambda x: 1 if x>=0 else 0)
kurs["up_avg_der"] = kurs["grad"].apply(lambda x: 1000 if x>=kurs["grad"].mean() else 0)
kurs["dupa"] = kurs["price"].apply(lambda x: x*1000-3000)



timespan = range(2002,2021)
hover = HoverTool()
hover.tooltips = [
("", "@legend"),
("Średni dochód na mieszkańca", "@y{int} zł"),
("rok", "@x"),
]
hover.formatters={'datetime': 'datetime'}

plots = []
legend = []
has_park = False
i = 0
for g in groups:
#     title = "Średnie dochodów na osobę w grupie gmin o gęstości zaludnienia: " + str(round(bins[i-1][0]))+" - "+str(round(bins[i-1][1]))

    title = "Średnie dochodów na osobę w gminach karpackich w grupie"+titles[i]
#     title = titles[i]
    graph = figure(title = title,
                   tools=[hover],
                   plot_width=900, plot_height=500) 
    graph.xaxis.axis_label = 'Rok'
    graph.yaxis.axis_label = 'Dochód [zł / osobę]'
    graph.toolbar.logo = None
    graph.toolbar_location = None
    no_park_avg = []
    is_park_avg = []
    park_name = []
    no_park_name = []
    for code in g:
        try:
            ts = dochod_wybrane_clean[dochod_wybrane_clean["Kod"] == code][[str(i) for i in timespan]].values
            x = [i for i in timespan]
            gestosc_ = gestosc[gestosc["Kod"] == code][year].values[0]
            procentp_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["ProcentGminyObjetyParkiemN"].values
            nazwa_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["Nazwa"].values[0]
            if wszystkie_gminy[wszystkie_gminy["Kod"] == code]["is_park"].values: 
                is_park_avg.append(list(ts)[0])
                park_name.append(nazwa_)
            else: 
                no_park_avg.append(list(ts)[0])
                no_park_name.append(nazwa_)
                
            if code in wazne_gminy: 
                data_ = ColumnDataSource({"x": x, "y": ts[0], 
                                 "legend": [nazwa_]*len(x),
                                })
                graph.line("x", "y", source=data_,
                           color = wazne_gminy_col.get(code),
                          line_width=1,
                           legend_label=nazwa_
                          )
        except:
            pass

    try:
        is_park_avg = np.array(is_park_avg)
        is_park_avg = [is_park_avg[:,i].mean() for i in range(is_park_avg[0].shape[0])]

        data_park = ColumnDataSource({"x": x, "y": is_park_avg, 
                         "legend": ["Gminy z parkami narodowymi"]*len(is_park_avg),
                        })
        graph.line("x", "y", source=data_park,
                  color = "green", 
                  line_width=1.5,
                      legend_label='Gminy parki'
                  )
    except:
        print("brak gmin z parkami")
        

    try:
        no_park_avg = np.array(no_park_avg)
        no_park_avg = [no_park_avg[:,i].mean() for i in range(no_park_avg[0].shape[0])]
        data_no_park = ColumnDataSource({"x": x, "y": no_park_avg, 
                         "legend": ["Gminy bez parków narodowych"]*len(no_park_avg),
                        })
        graph.line("x", "y", source=data_no_park,
                  color = "red", 
                  line_width=1.5,
                      legend_label='Gminy bez parków'
                  )
    except:
        print("brak gmin bez parków")        

    output_notebook()
    graph.add_layout(graph.legend[0], 'right')
    show(graph)
    # print(park_name)
    parki = wszystkie_gminy[(wszystkie_gminy["Kod"].isin(g))&(wszystkie_gminy["is_park"])]["Nazwa"].values
    df = wszystkie_gminy[wszystkie_gminy["Kod"].isin(g)].sort_values(by="is_park",ascending=False)[["Nazwa"]]
    df.index = [i for i in range(1,df.shape[0]+1)]
    df.rename(columns={"Nazwa": "Nazwa gminy"})
    def highlight_name(s, column):
        return ['background-color: green' if v in parki else 'background-color: red' for v in s]

    styled_table = df.style.apply(highlight_name, column=['Nazwa'], axis=1).hide_index().hide_columns()
    html = styled_table.render()

    i += 1



Loading BokehJS ...

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/2366084925.py:120: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



Loading BokehJS ...

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/2366084925.py:120: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



Loading BokehJS ...

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/2366084925.py:119: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/2366084925.py:120: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



In [10]:
display(HTML("<h1>" + "Gminy Dochody per capita do pokazywania" + "</h1>"))


dochod_wybrane_clean = dochod_wybrane.dropna()
timespan = range(2002,2021)

wazne_gminy = [1801083,#Ustrzyki Dolne
               1813012,#Bircza
               1813032]#Fredropol
wazne_gminy_col = {1801083:"turquoise", 1813012:"coral",1813032:"darkseagreen"}
rok = "2019"
park_mean = np.mean(dochod_wybrane_clean[ (dochod_wybrane_clean["Kod"].isin(wszystkie_gminy[wszystkie_gminy["is_park"]]["Kod"].values)) &(dochod_wybrane_clean["Kod"].isin(groups[0]))][rok].values)
no_park_mean = np.mean(dochod_wybrane_clean[ (dochod_wybrane_clean["Kod"].isin(wszystkie_gminy[~wszystkie_gminy["is_park"]]["Kod"].values)) &(dochod_wybrane_clean["Kod"].isin(groups[0]))][rok].values)

values = [park_mean,
          no_park_mean,
          dochod_wybrane_clean[dochod_wybrane_clean["Kod"] == wazne_gminy[0]][rok].values[0],
          dochod_wybrane_clean[dochod_wybrane_clean["Kod"] == wazne_gminy[1]][rok].values[0],
          dochod_wybrane_clean[dochod_wybrane_clean["Kod"] == wazne_gminy[2]][rok].values[0]]
names = ["Gminy parki", "Gminy bez parków","Ustrzyki Dolne","Bircza","Fredropol"]

from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.io import show
# from bokeh.models import CustomJS, Slider

# slider = Slider(start=2002, end=2020, value=2019, step=1, title="Rok")
# slider.js_on_change("value", CustomJS(code="""
#     console.log('slider: value=' + this.value, this.toString())
# """))

# show(slider)


source = ColumnDataSource(data=dict(names=names, values=values))

p = figure(x_range=names, height=250, tools="hover", tooltips="@names: @values{int} zł/ osobę",title="Porównanie dochodów per capita dla roku 2019"  )
p.vbar(x='names', top='values', width=0.9, source=source, 
       line_color='white', 
       fill_color=factor_cmap('names', palette=["green", "red", "turquoise", "coral", "darkseagreen"], factors=names)
      )

p.xgrid.grid_line_color = None
p.y_range.start = 0
show(p)

In [11]:
display(HTML("<h1>" + "Gminy % kanalizacji raczej nie do pokazywania" + "</h1>"))


timespan = range(2002,2021)
del graph
kanalizacja =kanalizacja.dropna()


wazne_gminy = [1801083,#Ustrzyki Dolne
               1813012,#Bircza
               1813032]#Fredropol
wazne_gminy_col = {1801083:"turquoise", 1813012:"coral",1813032:"darkseagreen"}
# titles = [" o najniższym udziale lasów na swoim terenie.",
#           " o średnim udziale lasów na swoim terenie.",
#           " o najwyższym udziale lasów na swoim terenie."]

timespan = range(2002,2020)
hover = HoverTool()
hover.tooltips = [
("", "@legend"),
("Kanalizacja", "@y{int} zł"),
("rok", "@x"),
]
hover.formatters={'datetime': 'datetime'}

plots = []
legend = []
has_park = False
i = 0
for g in groups:

    title = "Procent gospodarstw z dostępem do kanalizacyjnej w gminach karpackich w grupie"+titles[i]

    graph = figure(title = title,
                   tools=[hover],
                   plot_width=900, plot_height=500) 
    graph.xaxis.axis_label = 'Rok'
    graph.yaxis.axis_label = 'Dochód [zł / osobę]'
    graph.toolbar.logo = None
    graph.toolbar_location = None
    no_park_avg = []
    is_park_avg = []
    park_name = []
    no_park_name = []
    for code in g:
        try:
            ts = kanalizacja[kanalizacja["Kod"] == code][[str(i) for i in timespan]].values

            x = [i for i in timespan]
            gestosc_ = gestosc[gestosc["Kod"] == code][year].values[0]
            procentp_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["ProcentGminyObjetyParkiemN"].values
            nazwa_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["Nazwa"].values[0]
            if wszystkie_gminy[wszystkie_gminy["Kod"] == code]["is_park"].values: 
                is_park_avg.append(list(ts)[0])
                park_name.append(nazwa_)

            else: 
                no_park_avg.append(list(ts)[0])
                no_park_name.append(nazwa_)

            if code in wazne_gminy: 
                data_ = ColumnDataSource({"x": x, "y": ts[0], 
                                 "legend": [nazwa_]*len(x),

                                })
                graph.line("x", "y", source=data_,
                           color = wazne_gminy_col.get(code),
                          line_width=1,
                           legend_label=nazwa_
                          )
        except:
            pass

    try:
        is_park_avg = np.array(is_park_avg)
        is_park_avg = [is_park_avg[:,i].mean() for i in range(is_park_avg[0].shape[0])]

        data_park = ColumnDataSource({"x": x, "y": is_park_avg, 
                         "legend": ["Gminy z parkami narodowymi"]*len(is_park_avg),
                        })
        graph.line("x", "y", source=data_park,
                  color = "green", 
                  line_width=1.5,
                      legend_label='Gminy parki'
                  )

    except:
        print("brak gmin z parkami")
        

    try:
        no_park_avg = np.array(no_park_avg)
        no_park_avg = [no_park_avg[:,i].mean() for i in range(no_park_avg[0].shape[0])]

        data_no_park = ColumnDataSource({"x": x, "y": no_park_avg, 
                         "legend": ["Gminy bez parków narodowych"]*len(no_park_avg),
                        })
        graph.line("x", "y", source=data_no_park,
                  color = "red", 
                  line_width=1.5,
                      legend_label='Gminy bez parków'
                  )
    except:
        print("brak gmin bez parków")        

    output_notebook()
    graph.add_layout(graph.legend[0], 'right')
    graph.legend.click_policy="hide"
    show(graph)

    parki = wszystkie_gminy[(wszystkie_gminy["Kod"].isin(g))&(wszystkie_gminy["is_park"])]["Nazwa"].values
    df = wszystkie_gminy[wszystkie_gminy["Kod"].isin(g)].sort_values(by="is_park",ascending=False)[["Nazwa"]]
    df.index = [i for i in range(1,df.shape[0]+1)]
    df.rename(columns={"Nazwa": "Nazwa gminy"})
    def highlight_name(s, column):
        return ['background-color: green' if v in parki else 'background-color: red' for v in s]
    styled_table = df.style.apply(highlight_name, column=['Nazwa'], axis=1).hide_index().hide_columns()
    html = styled_table.render()
    i += 1

Loading BokehJS ...

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/105858006.py:129: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



Loading BokehJS ...

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/105858006.py:129: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



Loading BokehJS ...

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='index')`

/tmp/ipykernel_658453/105858006.py:128: FutureWarning:

this method is deprecated in favour of `Styler.hide(axis='columns')`

/tmp/ipykernel_658453/105858006.py:129: FutureWarning:

this method is deprecated in favour of `Styler.to_html()`



In [12]:
display(HTML("<h1>" + "Gminy % kanalizacji do pokazywania" + "</h1>"))

timespan = range(2002,2021)

wazne_gminy = [1801083,#Ustrzyki Dolne
               1813012,#Bircza
               1813032]#Fredropol
wazne_gminy_col = {1801083:"turquoise", 1813012:"coral",1813032:"darkseagreen"}
rok = "2019"
park_mean = np.mean(kanalizacja[ (kanalizacja["Kod"].isin(wszystkie_gminy[wszystkie_gminy["is_park"]]["Kod"].values)) &(kanalizacja["Kod"].isin(groups[0]))][rok].values)
no_park_mean = np.mean(kanalizacja[ (kanalizacja["Kod"].isin(wszystkie_gminy[~wszystkie_gminy["is_park"]]["Kod"].values)) &(kanalizacja["Kod"].isin(groups[0]))][rok].values)

values = [park_mean,
          no_park_mean,
          kanalizacja[kanalizacja["Kod"] == wazne_gminy[0]][rok].values[0],
          kanalizacja[kanalizacja["Kod"] == wazne_gminy[1]][rok].values[0],
          kanalizacja[kanalizacja["Kod"] == wazne_gminy[2]][rok].values[0]]
names = ["Gminy parki", "Gminy bez parków","Ustrzyki Dolne","Bircza","Fredropol"]

from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.io import show
# from bokeh.models import CustomJS, Slider

# slider = Slider(start=2002, end=2020, value=2019, step=1, title="Rok")
# slider.js_on_change("value", CustomJS(code="""
#     console.log('slider: value=' + this.value, this.toString())
# """))

# show(slider)


source = ColumnDataSource(data=dict(names=names, values=values))

p = figure(x_range=names, height=250, tools="hover", tooltips="@names: @values{int} zł/ osobę",title="Porównanie dostępu do kanalizacji dla roku 2019"  )
p.vbar(x='names', top='values', width=0.9, source=source, 
       line_color='white', 
       fill_color=factor_cmap('names', palette=["green", "red", "turquoise", "coral", "darkseagreen"], factors=names)
      )

p.xgrid.grid_line_color = None
p.y_range.start = 0
show(p)

In [13]:
aaa = [809480,838967,903353,972996,1046638,1147346,1262649,1330526,1395274,1508866,1562647,1591098,1647171,1735477,1790107,1907275,2033872,2199171,2242287]
aaa = np.sign(np.gradient(np.gradient(aaa)))
def detect_edges(series, legend):
    points = []
    for i in range(len(series)):
        if i == 0:
            points.append(timespan[i+1])
            continue
        elif i == len(series)-1:
            points.append(timespan[-1]+1)
            continue
        elif series[i] != series[i-1]:
            points.append(timespan[i])

    return[(points[s-1],points[s]) for s in range(len(points)) if s !=0  ] [1::2]

detect_edges(aaa,timespan)
# timespan

[(2008, 2010), (2011, 2013), (2018, 2021)]

In [30]:
display(HTML("<h1>" + "nie do pokazywania, ale wymaga sprawdzenia i ładnego objaśnienia" + "</h1>"))

#JAK PODAĆ, ŻE NA SPADEK WZROSTU PKB DOCHOD GMIN REAGUJE WZROSTEM??   WIDAĆ 1 ROK OPÓŹNIENIA
timespan = range(2002,2021)

# granice = [(2008, 2010), (2012, 2014), (2019, 2021)]#1 pochodna
# granice = [(2007, 2010), (2011, 2013), (2018, 2021)]#2 pochodna
granice = [(2008, 2010), (2012, 2014), (2019, 2021)]#1 pochodna
#detect_edges(pkb_p,timespan)

kurs = pd.read_csv("kurs_euro.csv")
kurs = kurs.groupby("year").agg("mean")
kurs["year"] = kurs.index
kurs = kurs[kurs["year"].isin(timespan)][["year","price"]]
kurs["grad"] = np.gradient(kurs.price.values)
kurs["rosnie"] = kurs["grad"].apply(lambda x: 1 if x>=0 else 0)
kurs["up_avg_der"] = kurs["grad"].apply(lambda x: 1000 if x>=kurs["grad"].mean() else 0)

timespan = range(2002,2021)

title = "Dochód narodowy brutto, poglądowo- skąd się wzięły te zakresy"

graph = figure(title = title,
               plot_width=900, plot_height=500) 
graph.xaxis.axis_label = 'Rok'
graph.yaxis.axis_label = ''
graph.toolbar.logo = None
graph.toolbar_location = None
pkb_pl1 = [779040,809480,838967,903353,972996,1046638,1147346,1262649,1330526,1395274,1508866,1562647,1591098,1647171,1735477,1790107,1907275,2033872,2199171,2242287] #GUS: roczne_wskazniki_makroekonomiczne_cz_iii.xlsx:  RACH_NARODOWE_ESA2010: Dochód narodowy brutto (ceny bieżące)
pkb_dif = [pkb_pl1[i] - pkb_pl1[i-1] for i in range(len(pkb_pl1)) if i!=0]
graph.line(kurs["year"].values.tolist(),pkb_dif, line_width=1.,legend_label = "diff")
graph.line(kurs["year"].values.tolist(),np.gradient(pkb_pl1), color ="red",legend_label = "1 pochodna")
graph.line(kurs["year"].values.tolist(),np.gradient(np.gradient(pkb_pl1)), color ="green",legend_label =" 2 pochodna")
# graph.line(kurs["year"].values.tolist(),[(i*20000) for i in pkb_p], color ="black",legend_label ="signum 2 pochodna")
for g in granice:
    graph.add_layout(BoxAnnotation(left=g[0]-1,right=g[1]-1, fill_alpha=0.1, fill_color='blue', line_color='blue'))
graph.add_layout(graph.legend[0], 'right')
show(graph)


/home/kwaks/gejopandas/lib/python3.10/site-packages/bokeh/models/sources.py:205: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 19), ('y', 20)

/home/kwaks/gejopandas/lib/python3.10/site-packages/bokeh/models/sources.py:205: BokehUserWarning:

ColumnDataSource's columns must be of the same length. Current lengths: ('x', 19), ('y', 20)



In [29]:
display(HTML("<h1>" + "Dochody per capita- nałożone okresy spowolnienia gospodarczego, do pokazywania?" + "</h1>"))

dochod_wybrane_clean = dochod_wybrane.dropna()
timespan = range(2002,2021)

# granice = detect_edges(pkb_p,timespan)


wazne_gminy = [1801083,#Ustrzyki Dolne
               1813012,#Bircza
               1813032]#Fredropol
wazne_gminy_col = {1801083:"turquoise", 1813012:"coral",1813032:"darkseagreen"}

kurs = pd.read_csv("kurs_euro.csv")
kurs = kurs.groupby("year").agg("mean")
kurs["year"] = kurs.index
kurs = kurs[kurs["year"].isin(timespan)][["year","price"]]
kurs["grad"] = np.gradient(kurs.price.values)
kurs["rosnie"] = kurs["grad"].apply(lambda x: 1 if x>=0 else 0)
kurs["up_avg_der"] = kurs["grad"].apply(lambda x: 1000 if x>=kurs["grad"].mean() else 0)


timespan = range(2002,2021)
hover = HoverTool()
hover.tooltips = [
("", "@legend"),
("Średni dochód na mieszkańca", "@y{int} zł"),
("rok", "@x"),
]
hover.formatters={'datetime': 'datetime'}

plots = []
legend = []
has_park = False
i = 0
for g in groups:

    title = "Średnie dochodów na osobę w gminach karpackich w grupie"+titles[i]

    graph = figure(title = title,
                   tools=[hover],
                   plot_width=900, plot_height=500) 
    graph.xaxis.axis_label = 'Rok'
    graph.yaxis.axis_label = 'Dochód [zł / osobę]'
    graph.toolbar.logo = None
    graph.toolbar_location = None
    no_park_avg = []
    is_park_avg = []
    park_name = []
    no_park_name = []
    for code in g:
        try:
            ts = dochod_wybrane_clean[dochod_wybrane_clean["Kod"] == code][[str(i) for i in timespan]].values
            x = [i for i in timespan]
            gestosc_ = gestosc[gestosc["Kod"] == code][year].values[0]
            procentp_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["ProcentGminyObjetyParkiemN"].values
            nazwa_ = wszystkie_gminy[wszystkie_gminy["Kod"] == code]["Nazwa"].values[0]
            if wszystkie_gminy[wszystkie_gminy["Kod"] == code]["is_park"].values: 
                is_park_avg.append(list(ts)[0])
                park_name.append(nazwa_)
            else: 
                no_park_avg.append(list(ts)[0])
                no_park_name.append(nazwa_)
                
            if code in wazne_gminy: 

                data_ = ColumnDataSource({"x": x, "y": ts[0], 
                                 "legend": [nazwa_]*len(x),
                                })
                graph.line("x", "y", source=data_,

                           color = wazne_gminy_col.get(code),
                          line_width=1,
                           legend_label=nazwa_
                          )
        except:
            pass

    try:
        is_park_avg = np.array(is_park_avg)
        is_park_avg = [is_park_avg[:,i].mean() for i in range(is_park_avg[0].shape[0])]

        data_park = ColumnDataSource({"x": x, "y": is_park_avg, 
                         "legend": ["Gminy z parkami narodowymi"]*len(is_park_avg),
                        })
        graph.line("x", "y", source=data_park,
                  color = "green", 
                  line_width=1.5,
                      legend_label='Gminy parki'
                  )
    except:
        print("brak gmin z parkami")
        

    try:
        no_park_avg = np.array(no_park_avg)
        no_park_avg = [no_park_avg[:,i].mean() for i in range(no_park_avg[0].shape[0])]

        data_no_park = ColumnDataSource({"x": x, "y": no_park_avg, 
                         "legend": ["Gminy bez parków narodowych"]*len(no_park_avg),
                        })
        graph.line("x", "y", source=data_no_park,
                  color = "red", 
                  line_width=1.5,
                      legend_label='Gminy bez parków'
                  )
    except:
        print("brak gmin bez parków")        

    for g in granice:
        graph.add_layout(BoxAnnotation(left=g[0]-1,right=g[1]-1, fill_alpha=0.1, fill_color='blue', line_color='blue'))


    output_notebook()
    graph.add_layout(graph.legend[0], 'right')
    show(graph)
    
    parki = wszystkie_gminy[(wszystkie_gminy["Kod"].isin(g))&(wszystkie_gminy["is_park"])]["Nazwa"].values
    df = wszystkie_gminy[wszystkie_gminy["Kod"].isin(g)].sort_values(by="is_park",ascending=False)[["Nazwa"]]
    df.index = [i for i in range(1,df.shape[0]+1)]
    df.rename(columns={"Nazwa": "Nazwa gminy"})
    def highlight_name(s, column):
        return ['background-color: green' if v in parki else 'background-color: red' for v in s]

#     styled_table = df.style.apply(highlight_name, column=['Nazwa'], axis=1).hide_index().hide_columns()
#     html = styled_table.render()
    i += 1

Loading BokehJS ...

Loading BokehJS ...

Loading BokehJS ...